### 0.환경설정

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import folium
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium
import json
from IPython.display import IFrame
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 20)

In [2]:
# 데이터프레임 로드

childcare_data = pd.read_csv('보육시설_위치_정보.csv', encoding="cp949")
df = pd.read_excel("노동데이터 최종_위도경도(19_22).xlsx")
df_clean = df.dropna()

##### 0-1. 데이터 전처리

In [3]:
# 필요인원수가 음수인 행 제거
df_clean = df_clean[df_clean['필요인원수'] >= 0]
df_clean = df_clean[df_clean['2022년'] <= 1]

# 불필요한 열 삭제
df_clean.drop(['2019년', '2020년', '2021년'], axis=1, inplace=True)

# Rename
df_clean.rename(columns = {'2022년':'노동수요비율', '위도':'지역위도', '경도':'지역경도'}, inplace=True)

# 위경도 0 drop
df_clean.drop(index = df_clean[df_clean['지역위도']==0].index, axis=1, inplace=True)

# '위도'와 '경도' 열 선택하여 2차원 배열로 변환
lat_lon = df_clean[['지역위도', '지역경도']].values

### 1. 지역 노동수치로 군집화

##### 1-1. 지역 기반 군집화

In [4]:
# KMeans 알고리즘을 사용하여 지역 위경도 기반 군집화 수행
lat_lon = df_clean[['지역위도', '지역경도']].values
kmeans_geo = KMeans(
    n_clusters=100, 
    random_state=42
).fit(lat_lon)

In [5]:
# 클러스터링된 결과를 데이터프레임에 추가
labels = kmeans_geo.labels_
centroids = kmeans_geo.cluster_centers_#.round(2)

In [6]:
df_clean['지역_군집'] = kmeans_geo.labels_
df_clean['지역_중심점_위도'] = [centroids[label][0] for label in labels]
df_clean['지역_중심점_경도'] = [centroids[label][1] for label in labels]

In [7]:
## 1-1 시각화

# 서울을 중심으로 하는 folium 지도 생성
map_center = [37.5665, 126.9780]
m = folium.Map(location=map_center, zoom_start=10)

geo_center = kmeans_geo.cluster_centers_
# 더 작은 군집 클러스터 중심점 표시
for center in geo_center:
    folium.Marker(
        center,
        popup='클러스터 중심점',
        icon=folium.Icon(color='green')
    ).add_to(m)

# HTML 파일로 저장
m.save('map_100_clusters.html')

In [8]:
# IFrame(src='map_100_clusters.html', width=700, height=600)

In [9]:
df_clean.drop(['구직합계', '구인합계', '지역위도', '지역경도', '시도', '시군구'], axis=1, inplace=True)

##### 1-2. 노동수치 기반 군집화

In [10]:
lat_lon = df_clean[['지역_군집', '노동수요비율']].values
kmeans_geo = KMeans(
    n_clusters=50,
    random_state=42
).fit(lat_lon)

# 데이터프레임에 추가
labels = kmeans_geo.labels_
centroids = kmeans_geo.cluster_centers_

df_clean['노동수치_군집'] = kmeans_geo.labels_
# 각 클러스터의 중심점 계산
for cluster_id in df_clean['노동수치_군집'].unique():
    # 현재 클러스터에 해당하는 데이터 추출
    cluster_data = df_clean[df_clean['노동수치_군집'] == cluster_id]
    # 클러스터에 속한 데이터포인트의 위도와 경도의 평균을 계산하여 중심점 좌표로 설정
    centroid_lat = cluster_data['지역_중심점_위도'].mean()
    centroid_lon = cluster_data['지역_중심점_경도'].mean()
    # 결과를 데이터프레임에 추가
    df_clean.loc[cluster_data.index, '노동수치_중심점_위도'] = centroid_lat
    df_clean.loc[cluster_data.index, '노동수치_중심점_경도'] = centroid_lon

In [11]:
# 각 클러스터의 중심점 계산
for cluster_id in df_clean['노동수치_군집'].unique():
    # 현재 클러스터에 해당하는 데이터 추출
    cluster_data = df_clean[df_clean['노동수치_군집'] == cluster_id]
    # 클러스터에 속한 데이터포인트의 위도와 경도의 평균을 계산하여 중심점 좌표로 설정
    centroid_lat = cluster_data['지역_중심점_위도'].mean()
    centroid_lon = cluster_data['지역_중심점_경도'].mean()
    # 결과를 데이터프레임에 추가
    df_clean.loc[cluster_data.index, '노동수치_중심점_위도'] = centroid_lat
    df_clean.loc[cluster_data.index, '노동수치_중심점_경도'] = centroid_lon

In [12]:
# df_clean['노동수치_군집'] = labels
# df_clean['중심점_위도'] = [centroids[label][0] for label in labels]
# df_clean['중심점_경도'] = [centroids[label][1] for label in labels]

In [13]:
# 서울을 중심으로 하는 folium 지도 생성
map_center = [37.5665, 126.9780]
m = folium.Map(location=map_center, zoom_start=10)

# 클러스터 중심점 표시
for index, row in df_clean.iterrows():
    folium.Marker(
        location=[row['노동수치_중심점_위도'], row['노동수치_중심점_경도']],  # 위도, 경도 순서로 리스트로 묶어줌
        popup=f'Cluster {row["노동수치_군집"]}',
        icon=folium.Icon(color='red')
    ).add_to(m)

# 결과를 HTML 파일로 저장
m.save("cluster_centers_map1000.html")

In [14]:
# IFrame(src='cluster_centers_map1000.html', width=700, height=600)

In [15]:
# df_clean.value_counts(['노동수치_군집','지역_군집','노동수치_중심점_위도','노동수치_중심점_경도']).sort_index().head(15)

In [16]:
# 시각화

In [17]:
df_clean.drop(['지역_군집'], axis=1, inplace=True)

### 2. 보육원 위치 기반 매칭

##### 2-1. 전처리

In [18]:
# 평균 노동수요비율
mean_values = df_clean.groupby('노동수치_군집')['노동수요비율'].mean()
df_clean['평균_노동수요비율'] = df_clean['노동수치_군집'].map(mean_values)

# df_clean.value_counts(['노동수치_군집', '평균_노동수요비율'])

In [19]:
# 군집_필요인원수 합

sum_values = df_clean.groupby('노동수치_군집')['필요인원수'].sum()
df_clean['필요인원수_합'] = df_clean['노동수치_군집'].map(sum_values)

# df_clean.value_counts(['필요인원수_합','노동수치_군집']).sort_index(ascending=False)

In [20]:
df_clean = df_clean[['노동수치_군집', '노동수치_중심점_위도', '노동수치_중심점_경도', '평균_노동수요비율', '필요인원수_합']]

시각화

In [21]:
import folium #folium 라이브러리 import
from geopy.geocoders import Nominatim # geopy.geocoders 라이브러리 import
geo_local = Nominatim(user_agent='South Korea') # South Korea로 geo_local 설정
from folium.plugins import HeatMap # folium.plugins 라이브러리 import

map_center = [36.5, 127.5] # 지도 센터값 설정
m = folium.Map(location=map_center, zoom_start=10) # 지도 객체 m 생성 (map_center로 중앙 설정 후, zoom 정도 7로 설정)

#반복 가능 객체를 만들고 추출 값을 리스트로 묶어서 heat_data 안에 넣기
heat_data = [[row['노동수치_중심점_위도'], row['노동수치_중심점_경도'], row['평균_노동수요비율'].round(1)] for _, row in df_clean.iterrows()] 
gradient = {
    0.9: 'green',
    0.8: 'green',
    0.7: 'orange',
    0.6: 'orange',
    0.5: 'red',
    0.4: 'red',
    0.3: 'darkred',
    0.2: 'darkred',
}

HeatMap(heat_data, gradient=gradient).add_to(m) # 생성된 HeatMap 객체를 Folium 지도 객체에 추가

heat_data2 = [[row['위도'], row['경도'], row['현원']] for _, row in childcare_data.iterrows()]

# lat_lon2 클러스터링
lat_lon2 = childcare_data[['위도', '경도']].values
# kmeans_geo2 = KMeans(n_clusters=100, random_state=42).fit(lat_lon2)
# geoCenter2 = kmeans_geo2.cluster_centers_

# 군집 클러스터 중심점 표시
for center in lat_lon2:
    folium.Circle(
        location=[center[0], center[1]],
        radius=100,
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(m)

m

In [22]:
# df_clean.value_counts(['필요인원수_합','노동수치_군집']).sort_index(ascending=False)

##### 2-2. 노동 배치 순위 산정

In [23]:
# 값 추출
mean_list = df_clean['평균_노동수요비율'].unique()

# 정렬
mean_list.sort()

# Mapping dictionary
number_dict = {}

# Mapping
for num,mean in enumerate(mean_list):
    number_dict[mean] = num
df_clean['배치'] = df_clean['평균_노동수요비율'].map(number_dict)

In [24]:
# df_clean.groupby(['노동수치_군집'])[['평균_노동수요비율', '필요인원수_합']].sum()

##### 2-3. 보육시설 위치 기반 거리 산출

In [25]:
# 거리 계산 함수
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반지름 (단위: km)
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) * np.sin(dlat / 2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) * np.sin(dlon / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

In [26]:
# df_clean.groupby(['노동수치_군집'])[['평균_노동수요비율', '필요인원수_합']].mean().sort_values(by='평균_노동수요비율').head()

In [27]:
# 보육 시설 데이터를 적절한 컬럼으로 읽어옵니다.
childcare_columns = ['시설명', '시(특별,광역)/도', '시/군/구', '위도', '경도', '현원']

In [28]:
# 보육 시설과 노동력 수치 데이터를 매칭하여 결과 데이터프레임을 생성하는 함수를 정의합니다.
def match_childcare_facilities(childcare_data, df_clean):
    matched_data = []

    # 각 노동수치 군집에 대해 반복합니다.
    for index, row in df_clean.iterrows():
        labour_location = (row['노동수치_중심점_위도'], row['노동수치_중심점_경도'])  # 해당 군집의 중심점 좌표를 가져옵니다.
        min_distance = float('inf')  # 초기 최소 거리를 무한대로 설정합니다.
        nearest_childcare = None  # 가장 가까운 보육 시설을 초기화합니다.

        # 모든 보육 시설에 대해 반복합니다.
        for idx, childcare_row in childcare_data.iterrows():
            childcare_location = (childcare_row['위도'], childcare_row['경도'])  # 보육 시설의 위치 좌표를 가져옵니다.
            distance = calculate_distance(labour_location[0], labour_location[1], childcare_location[0], childcare_location[1])  # 두 지점 사이의 거리를 계산합니다.

            # 현재 보육 시설이 이전까지의 최소 거리보다 더 가깝다면 업데이트합니다.
            if distance < min_distance:
                min_distance = distance
                nearest_childcare = childcare_row['시설명']
                nearest_childcare_num = childcare_row['현원']

        # 해당 군집과 가장 가까운 보육 시설 및 거리를 결과 리스트에 추가합니다.
        matched_data.append({'노동수치_군집': row['노동수치_군집'], '노동수치_중심점_위도': labour_location[0], '노동수치_중심점_경도': labour_location[1], '필요인원수_합':row['필요인원수_합'], '가장 가까운 보육 시설': nearest_childcare, '현원':nearest_childcare_num,  '거리 (km)': min_distance, '배치':row['배치']})

    # 결과 데이터프레임을 반환합니다.
    return pd.DataFrame(matched_data)

# 보육 시설과 노동력 수치 데이터를 매칭하여 결과 데이터프레임을 생성합니다.
matched_childcare_data = match_childcare_facilities(childcare_data, df_clean)
matched_childcare_data

# 매칭 결과를 출력합니다.
matched_childcare_data.groupby('노동수치_군집')[['필요인원수_합', '가장 가까운 보육 시설', '현원', '거리 (km)', '배치']].sum()

,필요인원수_합,가장 가까운 보육 시설,현원,거리 (km),배치
노동수치_군집,,,,,
0.0,19938.0,요엘원요엘원요엘원,120,72.267664,81.0
1.0,202158.0,새볕원새볕원새볕원,63,7.830771,30.0
2.0,480684.0,영실애육원영실애육원,82,60.950513,48.0
3.0,134400.0,충북육아원충북육아원충북육아원,123,3.545389,51.0
4.0,463230.0,선한사마리아원선한사마리아원선한사마리아원선한사마리아원선한사마리아원,185,37.855237,140.0
...,...,...,...,...,...
45.0,5828.0,고성애육원,23,0.000000,12.0
46.0,59752.0,전주영아원전주영아원,70,1.714755,90.0
47.0,12666.0,광주나자렛집광주나자렛집,24,3.322291,52.0


In [29]:
# 매칭 결과 확인
matched_result = match_childcare_facilities(childcare_data, df_clean)

In [34]:
df_clean

,노동수치_군집,노동수치_중심점_위도,노동수치_중심점_경도,평균_노동수요비율,필요인원수_합,배치
1,4,37.168294,127.181125,0.551050,92646,28
2,4,37.168294,127.181125,0.551050,92646,28
3,4,37.168294,127.181125,0.551050,92646,28
18,8,36.424102,127.578841,0.672693,12155,38
21,17,37.456202,127.010183,0.712920,29238,42
...,...,...,...,...,...,...
253,32,36.574985,127.675123,0.465152,41592,13
254,29,35.512108,127.364154,0.351039,28897,6
255,45,34.973278,128.322233,0.450254,5828,12
258,20,35.831563,127.240081,0.417006,65780,9


In [41]:
# 노동수치 군집 기준으로 가까운 보육원 3개 찾기
# 군집 번호, 군집 필요인원수_합, 보육원 3개, 보육원 현원
def find_nearest_childcare_centers(df_clean, childcare_data, n=3):
    nearest_childcare_centers = []

    # 각 노동수치 군집에 대해 반복
    for cluster_id in sorted(list(df_clean['노동수치_군집'].unique())):
        # 현재 클러스터에 해당하는 데이터 추출
        cluster_data = df_clean[df_clean['노동수치_군집'] == cluster_id]

        # 현재 클러스터의 중심점 좌표
        centroid_lat = cluster_data['노동수치_중심점_위도'].iloc[0]
        centroid_lon = cluster_data['노동수치_중심점_경도'].iloc[0]

        # 현재 클러스터의 노동수치 군집 번호
        cluster_number = cluster_data['노동수치_군집'].iloc[0]

        # print(centroid_lat, centroid_lon, cluster_number)

        # 현재 클러스터의 중심점과 가장 가까운 보육 시설 찾기
        distances = []  # 각 보육 시설까지의 거리를 저장할 리스트
        for index, childcare_row in childcare_data.iterrows():
            childcare_location = (childcare_row['위도'], childcare_row['경도'])
            distance = calculate_distance(centroid_lat, centroid_lon, childcare_location[0], childcare_location[1])
            distances.append((childcare_row['시설명'], distance))

        # 거리가 가장 짧은 보육 시설 n개 선택
        nearest_childcare_centers.extend(sorted(distances, key=lambda x: x[1])[:n])

        # 결과에 노동수치 군집 번호 추가
        nearest_childcare_centers[-n:] = [(cluster_number, childcare, distance) for childcare, distance in nearest_childcare_centers[-n:]]
        print(nearest_childcare_centers)

        # 군집 번호가 바뀔 때마다 줄 바꿈 추가
        nearest_childcare_centers.append('\n')

    return nearest_childcare_centers

In [42]:
# 노동수치 군집 번호를 기준으로 가장 가까운 보육원 3개 찾기
nearest_childcare_centers = find_nearest_childcare_centers(df_clean, childcare_data, n=3)

# 결과 출력
for item in nearest_childcare_centers:
    if isinstance(item, str) and item == '\n':
        print()  # 줄 바꿈 출력
    else:
        cluster_number, childcare_center, distance = item
        print(f"노동수치 군집 번호: {cluster_number}, 보육 시설: {childcare_center}, 거리: {distance} km")

[(0, '요엘원', 24.089221274216367), (0, '희망샘학교', 24.94755903155645), (0, '고창행복원', 27.74589784029942)]
[(0, '요엘원', 24.089221274216367), (0, '희망샘학교', 24.94755903155645), (0, '고창행복원', 27.74589784029942), '\n', (1, '새볕원', 2.610257147941768), (1, '천광원', 7.769941304234395), (1, '신애보육원', 8.362211749332868)]
[(0, '요엘원', 24.089221274216367), (0, '희망샘학교', 24.94755903155645), (0, '고창행복원', 27.74589784029942), '\n', (1, '새볕원', 2.610257147941768), (1, '천광원', 7.769941304234395), (1, '신애보육원', 8.362211749332868), '\n', (2, '영실애육원', 30.47525663111934), (2, '애향아동복지원', 33.0044386407727), (2, '상주보육원', 34.66693383169845)]
[(0, '요엘원', 24.089221274216367), (0, '희망샘학교', 24.94755903155645), (0, '고창행복원', 27.74589784029942), '\n', (1, '새볕원', 2.610257147941768), (1, '천광원', 7.769941304234395), (1, '신애보육원', 8.362211749332868), '\n', (2, '영실애육원', 30.47525663111934), (2, '애향아동복지원', 33.0044386407727), (2, '상주보육원', 34.66693383169845), '\n', (3, '충북육아원', 1.1817962935905002), (3, '충북혜능보육원', 1.7776400700199908), (3, '현양원', 3.